# 흑백(Raw) 디지털 영상처리 및 분석 프로그램

### OpenCV를 사용하지 않고 직접 함수를 작성하여 구현. C++ 알고리즘 코드를 참조하였음.

### 프로젝트 요약
* 개발 도구: Python
* 주요 라이브러리: math, tkinter, operator, threading, struct

![](https://github.com/rosa-yuri/my_projects/blob/master/rawppt1.png)
![](https://github.com/rosa-yuri/my_projects/blob/master/rawppt2.png)
![](https://github.com/rosa-yuri/my_projects/blob/master/rawppt3.png)

In [ ]:
import os.path
import math
from tkinter.simpledialog import *
from tkinter.filedialog import *
import threading
import struct

#함수선언 부분

##파일입출력
def openFile():
    global window, canvas, paper, filename, inImageR, inImageG, inImageB, outImageR, outImageG, outImageB, inW, inH, outW, outH
    filename = askopenfilename(parent=window,
                               filetypes=(("RAW파일", "*.raw"), ("모든파일", "*.*")))
    loadImage(filename)  # 파일 --> 입력메모리
    equal()  # 입력메모리--> 출력메모리

def saveFile() :
    global window, canvas, paper, filename,inImage, outImage,inW, inH, outW, outH
    saveFp = asksaveasfile(parent=window, mode='wb', defaultextension="*.raw", filetypes=(("RAW파일", "*.raw"), ("모든파일", "*.*")))
    for i in range(outH):
        for k in range(outW):
            saveFp.write(struct.pack('B',outImage[i][k]))
    saveFp.close()

def exitFile():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    window.quit()
    window.destroy()

def loadImage(fname):
    global window, canvas, paper, photo, filename, inImage, outImage, inW, inH, outW, outH
    fsize = os.path.getsize(fname)  # 파일 크기 확인
    inH = inW = int(math.sqrt(fsize))  # 입력메모리 크기 결정! (중요)
    inImage = []
    for i in range(inH):  # 입력메모리 확보(0으로 초기화)
        tmpList = []
        for k in range(inW):
            tmpList.append(0)
        inImage.append(tmpList)
    # 파일 --> 메모리로 데이터 로딩
    fp = open(fname, 'rb')  # 파일 열기(바이너리 모드)
    for i in range(inH):
        for k in range(inW):
            inImage[i][k] = int(ord(fp.read(1)))
    fp.close()

def display() :
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH, paper_copy
    # 기존에 캐버스 있으면 뜯어내기.
    if  canvas != None :
        canvas.destroy()
    # 화면 준비 (고정됨)
    # window.geometry(str(outH) + 'x' + str(outW))
    canvas = Canvas(window, width=outW, height=outH)
    paper = PhotoImage(width=outW, height=outH)
    canvas.create_image((outW/2, outH/2), image=paper, state='normal')
        # 화면에 출력
    def putPixel() :
        for i in range(0, outH) :
            for k in range(0, outW) :
                data = outImage[i][k]
                paper.put('#%02x%02x%02x' % (data, data, data), (k,i))
    threading.Thread(target=putPixel).start()
    canvas.pack()

##화소점처리:(Pixel Processing): 동일영상, 화소값반전, 밝게하기(덧셈연산), 어둡게(뺄셈), 곱셈, 나눗셈, 파라볼라(Cap, Cup), 감마, 이진화, 특정범위 추출
def equal() :  # 동일 영상 알고리즘
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요! 출력메모리의 크기를 결정
    outW = inW
    outH = inH
    outImage = []
    for i in range(outH):  # 출력메모리 확보(0으로 초기화)
        tmpList = []
        for k in range(outW):
            tmpList.append(0)
        outImage.append(tmpList)
    #############################
    # 진짜 영상처리 알고리즘을 구현
    ############################
    for  i  in  range(inH) :
        for  k  in  range(inW) :
            outImage[i][k] = inImage[i][k]
    display()

def reverse():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH

    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    for i in range(inH):
        for k in range(inW):
            outImage[i][k] = 255 - inImage[i][k]
    display()

def brightAdd():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH

    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    value = askinteger('밝게하기', '값(1~255)을 입력하세요:', minvalue=1, maxvalue=255)
    for i in range(inH):
        for k in range(inW):
            if inImage[i][k] + value > 255:
                outImage[i][k] = 255
            else:
                outImage[i][k] = inImage[i][k] + value
    display()

def brightSub():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH

    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    value = askinteger('어둡게하기', '값(1~255)을 입력하세요:', minvalue=1, maxvalue=255)
    for i in range(inH):
        for k in range(inW):
            if inImage[i][k] - value < 0:
                outImage[i][k] = 0
            else:
                outImage[i][k] = inImage[i][k] - value
    display()

def mulitply(): #더욱 밝게(곱셈)
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH
    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    value = askinteger('더욱 밝게하기', '값(1~10)을 입력하세요:', minvalue=1, maxvalue=10)
    for i in range(inH):
        for k in range(inW):
            if inImage[i][k] * value > 255:
                outImage[i][k] = 255
            elif inImage[i][k] * value < 0:
                outImage[i][k] = 0
            else:
                outImage[i][k] = int(inImage[i][k] * value)
    display()

def division(): #더욱 어둡게(나눗셈)
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH
    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    value = askinteger('더욱 어둡게하기', '값(1~10)을 입력하세요:', minvalue=1, maxvalue=10)
    for i in range(inH):
        for k in range(inW):
            if inImage[i][k] / value > 255:
                outImage[i][k] = 255
            elif inImage[i][k] / value < 0:
                outImage[i][k] = 0
            else:
                outImage[i][k] = int(inImage[i][k] / value)
    display()

def gamma():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH

    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    value = askfloat('감마처리', '실수값(0~1)을 입력하세요:', minvalue=0, maxvalue=1)
    for i in range(inH):
        for k in range(inW):
            new_value = pow(inImage[i][k], 1 / value)
            if new_value < 0:
                outImage[i][k] = 0
            elif new_value > 255:
                outImage[i][k] = 255
            else:
                outImage[i][k] = int(new_value)
    display()

def cap():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH

    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    for i in range(inH):
        for k in range(inW):
            new_value = 255 - 255 * pow((getdouble)(inImage[i][k] / 128.0) - 1.0, 2)
            if new_value < 0:
                outImage[i][k] = 0
            elif new_value > 255:
                outImage[i][k] = 255
            else:
                outImage[i][k] = int(new_value)
    display()

def cup():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH

    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    for i in range(inH):
        for k in range(inW):
            new_value = 255 * pow((getdouble)(inImage[i][k] / 128.0) - 1.0, 2)
            if new_value < 0:
                outImage[i][k] = 0
            elif new_value > 255:
                outImage[i][k] = 255
            else:
                outImage[i][k] = int(new_value)
    display()

def binarAdaptive():    #적응 이진화
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH

    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################
    value = askfloat('이진화 처리', '숫자(1~255)를 입력하세요.', minvalue=1, maxvalue=255)
    for i in range(inH):
        for k in range(inW):
            if outImage[i][k] >= value:
                outImage[i][k] = 255
            else:
                outImage[i][k] = 0
    display()

def spotLight():  #범위 강조 변환
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요!! 출력메모리 크기 결정
    outW = inW
    outH = inH
    for i in range(outH):  # 입력 메모리 확보(0으로 초기화)
        tmplist = []
        for k in range(outW):
            tmplist.append(0)  # 0으로 초기화
        outImage.append(tmplist)
    #####################################
    ##  이제부터 진짜 영상처리 알고리즘 구현 ##
    ####################################

    startPoint = askinteger('강조할 범위 값', '시작 값(0~255):', minvalue=0, maxvalue=255)
    endPoint = askinteger('강조할 범위 값', '마지막 값(0~255):', minvalue=0, maxvalue=255)
    for i in range(inH):
        for k in range(inW):
            if (inImage[i][k] >= startPoint) & (inImage[i][k] <= endPoint):
                outImage[i][k] = 255
            else:
                outImage[i][k] = inImage[i][k]
    display()

##기하학 처리(Geometry Processing): 위치변경, 상하반전, 좌우반전, 이동, 회전, 확대, 축소, 영상 합성
def upDown() :  # 상하반전
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요! 출력메모리의 크기를 결정
    outW = inW
    outH = inH
    outImage = []
    for i in range(outH):  # 출력메모리 확보(0으로 초기화)
        tmpList = []
        for k in range(outW):
            tmpList.append(0)
        outImage.append(tmpList)
    #############################
    # 진짜 영상처리 알고리즘을 구현
    ############################
    for  i  in  range(inH) :
        for  k  in  range(inW) :
            outImage[outW-1-i][k] = inImage[i][k]
    display()

def rightLeft():    #좌우반전
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요! 출력메모리의 크기를 결정
    outW = inW
    outH = inH
    outImage = []
    for i in range(outH):  # 출력메모리 확보(0으로 초기화)
        tmpList = []
        for k in range(outW):
            tmpList.append(0)
        outImage.append(tmpList)
    #############################
    # 진짜 영상처리 알고리즘을 구현
    ############################
    for  i  in  range(inH) :
        for  k  in  range(inW) :
            outImage[i][outH-1-k] = inImage[i][k]
    display()

##화면이동 3가지 함수
def panImage(): #화면이동 스위치
    global panYN
    panYN = True

def mouseClick(event) :
    global window, panYN, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH, sx, sy, ex, ey
    if not panYN :
        return
    sx = event.x
    sy = event.y

def mouseDrop(event):
    global window, panYN, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH, sx, sy, ex, ey
    if not panYN:
        return
    ex = event.x
    ey = event.y
    mx = ex - sx
    my = ey - sy

    # 중요! 출력메모리의 크기를 결정
    outW = inW
    outH = inH
    outImage = []
    for i in range(outH):  # 출력메모리 확보(0으로 초기화)
        tmpList = []
        for k in range(outW):
            tmpList.append(0)
        outImage.append(tmpList)
    #############################
    # 진짜 영상처리 알고리즘을 구현
    ############################
    for  i  in  range(inH) :
        for  k  in  range(inW) :
            if 0 <= i-mx < outH and 0 <= k-my < outW:
                outImage[i-mx][k-my] = inImage[i][k]
    panYN =False
    display()

def zoomIn():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요! 출력메모리의 크기를 결정
    scale = askinteger('확대하기', '값(2~32)을 입력하세요:', minvalue=2, maxvalue=32)
    outW = int(inW * scale)
    outH = int(inH * scale)
    outImage = []
    for i in range(outH):  # 출력메모리 확보(0으로 초기화)
        tmpList = []
        for k in range(outW):
            tmpList.append(0)
        outImage.append(tmpList)
    #############################
    # 진짜 영상처리 알고리즘을 구현
    ############################
    for i in range(outH):
        for k in range(outW):
            outImage[i][k] = inImage[int(i / scale)][int(k / scale)]
    display()

def zoomOut():
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    # 중요! 출력메모리의 크기를 결정
    scale = askinteger('축소하기', '값(2~32)을 입력하세요:', minvalue=2, maxvalue=32)
    outW = int(inW / scale)
    outH = int(inH / scale)
    outImage = []
    for i in range(outH):  # 출력메모리 확보(0으로 초기화)
        tmpList = []
        for k in range(outW):
            tmpList.append(0)
        outImage.append(tmpList)
    #############################
    # 진짜 영상처리 알고리즘을 구현
    ############################
    for i in range(inH):
        for k in range(inW):
            outImage[int(i / scale)][int(k / scale)] = inImage[i][k]
    display()

##이미지 데이터분석
def a_average():    #평균값
    global window, canvas, paper, filename, inImage, outImage, inW, inH, outW, outH
    rawSum = 0
    for i in range(inH):
        for k in range(inW):
            rawSum += inImage[i][k]
    inRawAvg = int(rawSum / (inW * inH))
    rawSum = 0
    for i in range(outH):
        for k in range(outW):
            rawSum += outImage[i][k]
    outRawAvg = int(rawSum / (outW * outH))

    subWindow = Toplevel(window)  # 부모(window)에 종속된 자식(sub-window)
    subWindow.geometry(str(outH) + 'x' + str(outW))
    label1 = Label(subWindow, text='입력영상 평균값-->' + str(inRawAvg))
    label2 = Label(subWindow, text='출력영상 평균값-->' + str(outRawAvg))
    label1.pack()
    label2.pack()
    subWindow.mainloop()

#변수선언
sx, sy, ex, ey = [0]*4
window, canvas, paper, filename = [None] * 4
inImage, outImage = [], []
inW, inH, outW, outH = [0] * 4
x1, y1, x2, y2 = None, None, None, None
penWidth, penColor = 5, 'black'

#메인코드
if __name__ == '__main__':
    window = Tk()
    window.title('Min.Project(); Digital Image Analyzer ver.0.05')
    window.geometry('800x500')
    window.bind("<Button-1>", mouseClick)
    window.bind("<ButtonRelease-1>", mouseDrop)

    canvas = Canvas(window, height = 300, width=300)

    canvas.pack()

    ##사진 공간 만들기
    photo = PhotoImage()
    plabel1=Label(window, image=photo)
    plabel2=Label(window, image=photo)
    plabel1.pack(expand=1, anchor=W)
    plabel2.pack(expand=1, anchor=E)

    ##메뉴 만들기
    mainMenu = Menu(window)
    window.config(menu=mainMenu)

    fileMenu = Menu(mainMenu)
    mainMenu.add_cascade(label='파일', menu=fileMenu)
    fileMenu.add_command(label='GrayScale열기', command=openFile)
    fileMenu.add_command(label='GrayScale저장', command=saveFile)
    fileMenu.add_separator()
    fileMenu.add_command(label='종료', command=exitFile)

    pixelMenu = Menu(mainMenu)
    mainMenu.add_cascade(label='화소점처리', menu=pixelMenu)
    pixelMenu.add_command(label='gray동일영상', command=equal)
    pixelMenu.add_command(label='화소값반전', command=reverse)
    pixelMenu.add_command(label='밝게(덧셈)', command=brightAdd)
    pixelMenu.add_command(label='어둡게(뺄셈)', command=brightSub)
    pixelMenu.add_command(label='더 밝게(곱셈)', command=mulitply)
    pixelMenu.add_command(label='더 어둡게(나눗셈)', command=division)
    pixelMenu.add_command(label='파라볼라(Cap)', command=cap)
    pixelMenu.add_command(label='파라볼라(Cup)', command=cup)
    pixelMenu.add_command(label='감마', command=gamma)
    pixelMenu.add_command(label='적응이진화', command=binarAdaptive)
    pixelMenu.add_command(label='범위강조변환', command=spotLight)

    geoMenu = Menu(mainMenu)
    mainMenu.add_cascade(label='기하학처리', menu=geoMenu)
    geoMenu.add_command(label='상하반전', command=upDown)
    geoMenu.add_command(label='좌우반전', command=rightLeft)
    geoMenu.add_command(label='화면이동', command=panImage)
    geoMenu.add_command(label='확면축소', command=zoomOut)
    geoMenu.add_command(label='확면확대(역방향)', command=zoomIn)

    analyzeMenu = Menu(mainMenu);mainMenu.add_cascade(label='데이터분석', menu=analyzeMenu)
    analyzeMenu.add_command(label='평균값', command=a_average)


    window.mainloop()